# analisis

> Implementaciones de métodos y algoritmos para analizar el uso de suelo

In [ ]:
#| default_exp analisis

In [2]:
#| hide
from nbdev.showdoc import *

In [12]:
#| export
from fastcore.basics import *
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon
from geocube.api.core import make_geocube
from mlxtend.frequent_patterns import apriori
import numpy as np
from pyLandUseMX.descargas import *
from pyLandUseMX.denue import *
from pyLandUseMX.mallas import *


In [2]:
%load_ext autoreload
%autoreload 2

## Market Basket

El análisis de _Canasta de Mercado_ (o análisis de afinidad) busca encontrar patrones de co-ocurrencia... [EXPLICAR MÁS]


Vamos a cagar los datos de ejemplo del DENUE, agregarlos por categoría y luego seleccionar únicamente la categoría de _Comercio_

In [7]:
descarga_denue('ejemplo')
denue = Denue.desde_archivo("datos/descargas/ejemplo_denue.gpkg")
categorias = {
    'Manufacturas': ['^31.*5$', '^32.*5$', '^33.*5$'],
    'Oficinas': ['^51', '^521', '^523', '^524', '^5312', '^5313', '^541', '^55'],
    'Comercio': ['^46[123456]']
}
usos = denue.agrega_en_usos(categorias)
comercio = usos.datos.loc[usos.datos.Categoria == 'Comercio']
comercio.head()

El archivo ya está descargado


,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipoCenCom,cve_ent,cve_mun,cve_loc,ageb,...,index_right,OBJECTID,Shape_Leng,NOM_CIUDAD,Shape_Le_1,Shape_Area,CVE_SUN,SUN,geometry,Categoria
1,TIENDA NATURISTA EL ARTE DE LA NATURALEZA,None,464113,"Comercio al por menor de productos naturistas,...",0 a 5 personas,None,09,007,0001,1814,...,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-99.06312 19.33782),Comercio
3,SIN NOMBRE,None,461130,Comercio al por menor de frutas y verduras fre...,0 a 5 personas,None,09,008,0001,0423,...,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-99.25436 19.30129),Comercio
6,ABARROTES LA TIA,None,461110,"Comercio al por menor en tiendas de abarrotes,...",0 a 5 personas,None,15,121,0001,1763,...,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-99.19269 19.58976),Comercio
7,BIZUTERIA SIN NOMBRE,None,463215,Comercio al por menor de bisutería y accesorio...,0 a 5 personas,None,15,122,0001,0847,...,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-98.94058 19.31219),Comercio
8,FRUTAS Y VERDURAS SIN NOMBRE,None,461130,Comercio al por menor de frutas y verduras fre...,0 a 5 personas,None,15,060,0001,1037,...,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-99.33855 19.61941),Comercio


Construimos una malla

In [10]:
malla = Malla.desde_capa(comercio.to_crs(32614), 1000)

Agregamos los puntos en la malla de acuerdo a las claves SCIAN

In [11]:
agregado = malla.agrega_puntos(comercio, clasificacion='codigo_act')
agregado.malla

,grid_id,461110,461121,461122,461123,461130,461140,461150,461160,461170,...,466114,466211,466212,466311,466312,466313,466314,466319,466410,geometry
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((449355.857 2095991.537, 450355.857 2..."
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((449355.857 2096991.537, 450355.857 2..."
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((449355.857 2097991.537, 450355.857 2..."
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((449355.857 2098991.537, 450355.857 2..."
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((449355.857 2099991.537, 450355.857 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10255,10255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((538355.857 2204991.537, 539355.857 2..."
10256,10256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((538355.857 2205991.537, 539355.857 2..."
10257,10257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((538355.857 2206991.537, 539355.857 2..."
10258,10258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((538355.857 2207991.537, 539355.857 2..."


In [13]:
apriori(agregado.malla, min_support=0.6, use_colnames=True)

/home/plablo/miniconda3/envs/landuse/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value POLYGON ((449355.8569789248 2095991.5366673418, 450355.8569789248 2095991.5366673418, 450355.8569789248 2096991.5366673418, 449355.8569789248 2096991.5366673418, 449355.8569789248 2095991.5366673418))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()